# Time integrators

A collection of efficient time integration methods for the damped oscillator with optional nonlinearities and initial conditions or excitation inputs.

In [ ]:
# | default_exp time_integrators

In [ ]:
# | exporti

from collections.abc import Callable
from functools import partial

import einops
import jax
import jax.numpy as jnp

## Stormer-verlet

The stormer-verlet is a centered difference scheme used to approximate derivatives. In the present case we use it for approximating the second and first derivatives of an oscillator defined by the following differential equation:

$$
\ddot{q} + c \dot{q} + k q = f(t)
$$

The finite difference operators are defined as:

$$
\begin{align}
\delta_t q &= \frac{q^{n+1} - q^{n-1}}{2 h} \\
\delta_{tt} q &= \frac{q^{n+1} - 2 q^n + q^{n-1}}{h^2}
\end{align}
$$

for the first derivative and the second derivative respectively. The difference equation for the oscillator is then:

$$
\delta_{tt} q + c \delta_t q + k q = f(t)
$$

after expanding and some algebraic manipulation to isolate $q^{n+1}$ we get:


$$
\bigl(\tfrac{1}{h^2} + \tfrac{c}{2h}\bigr)\,q^{n+1} +
\bigl(-\tfrac{2}{h^2} + k\bigr) q^n +
\bigl(\tfrac{1}{h^2} - \tfrac{c}{2h}\bigr)\,q^{n-1} =
f(t^n).
$$

$$
\boxed{
q^{n+1} = \frac{2 h^2}{2 + c h}
\Bigl[
f(t^n) + \Bigl(\tfrac{2}{h^2} - k\Bigr) q^n +
\Bigl(-\tfrac{1}{h^2} + \tfrac{c}{2h}\Bigr) q^{n-1}
\Bigr].
}
$$

To make it more readable we can define the following constants:

$$
\begin{align}
a &= \frac{2 h^2}{2 + c h} \\
b &= \frac{2}{h^2} - k \\
c &= -\frac{1}{h^2} + \frac{c}{2h}
\end{align}
$$

In [ ]:
# | export
# | hide


def A_inv_vector(
    h: float,  # temporal grid spacing
    damping: jnp.ndarray,  # damping term vector
) -> jnp.ndarray:
    return 2.0 * h**2 / (2.0 + damping * h)


def B_vector(
    h,  # temporal grid spacing (scalar)
    stiffness,  # stiffness term (vector)
):
    return 2.0 / h**2 - stiffness


def C_vector(
    h,  # temporal grid spacing (scalar)
    damping,  # damping term (vector)
):
    return -1.0 / h**2 + (damping / (2.0 * h))

In [ ]:
# | export


def make_identity_nl_fn():
    def nl_fn(q):
        return q

    return nl_fn


def make_vk_nl_fn(H):
    """
    Returns a function that computes nl given q.
    """

    def nl_fn(q):
        return einops.einsum(
            H,
            H,
            q,
            q,
            q,
            "n p q, n r s, p, q, r -> s",
        )

    return nl_fn


def make_tm_nl_fn(lambda_mu, factors):
    """
    Returns a function that computes nl given q.
    """

    def nl_fn(q):
        return lambda_mu * q * (factors @ q**2)

    return nl_fn

In [ ]:
# | export


def plate_tau_with_density(plate_params):
    plate_tau = (plate_params.E * plate_params.h) / (
        2 * plate_params.l1 * plate_params.l2 * (1 - plate_params.nu**2)
    )
    return plate_tau / plate_params.density


def string_tau_with_density(string_params):
    string_tau = (
        string_params.E * string_params.A / (string_params.length * 2)
    ) / string_params.density
    return string_tau

In [ ]:
# | export


def rk4_step(
    u0: jnp.ndarray,  # initial conditions (n_modes,)
    v0: jnp.ndarray,  # initial conditions (n_modes,)
    dt: float,  # time step
    gamma2_mu: jnp.ndarray,  # damping (n_modes,)
    omega_mu_squared: jnp.ndarray,  # frequency (n_modes,)
):
    """
    One step of RK4 for the second-order damped oscillator.
    Returns: (u1, v1)
    """
    n_modes = u0.shape[0]

    def f(x):
        u = x[:n_modes]
        v = x[n_modes:]
        return jnp.concatenate([v, -gamma2_mu * v - omega_mu_squared * u])

    x = jnp.concatenate([u0, v0])
    k1 = f(x)
    k2 = f(x + 0.5 * dt * k1)
    k3 = f(x + 0.5 * dt * k2)
    k4 = f(x + dt * k3)

    x_next = x + (dt / 6) * (k1 + 2 * k2 + 2 * k3 + k4)

    u1 = x_next[:n_modes]
    v1 = x_next[n_modes:]
    return u1, v1


def second_order_step(
    u0: jnp.ndarray,  # initial conditions (n_modes,)
    v0: jnp.ndarray,  # initial conditions (n_modes,)
    dt: float,  # time step
    gamma2_mu: jnp.ndarray,  # damping (n_modes,)
    omega_mu_squared: jnp.ndarray,  # frequency (n_modes,)
):
    """
    One step of 2nd-order Taylor expansion for damped oscillator.
    Returns: (u1, v1)
    """
    a = -gamma2_mu * v0 - omega_mu_squared * u0  # ddot(q)
    u1 = u0 + v0 * dt + 0.5 * a * dt**2
    v1 = v0 + a * dt
    return u1, v1

In [ ]:
# | export


@partial(jax.jit, static_argnames=("nl_fn",))
def solve_sv_vk_jax_scan(
    A_inv: jnp.ndarray,
    B: jnp.ndarray,
    C: jnp.ndarray,
    modal_excitation: jnp.ndarray,  # (T, n_modes)
    nl_fn: Callable,
):
    n_modes = A_inv.shape[0]
    q = jnp.zeros((n_modes,))  # Modal displacement vector at n (n_modes, 1)
    q_prev = jnp.zeros((n_modes,))  # Modal displacement vector at n-1

    def advance_state(
        state: jnp.ndarray,  # inital state
        x: jnp.ndarray,  # input
    ) -> tuple[jnp.ndarray, jnp.ndarray]:  # carry, output
        # unpack state
        q_prev, q = state

        nl = nl_fn(q)

        # compute the next state
        q_next = B * q + C * q_prev - A_inv * nl + x

        # return the next state and the output
        return (q, q_next), q_next

    state, final = jax.lax.scan(
        advance_state,
        (q_prev, q),
        modal_excitation,  # (T, n_modes)
        unroll=8,
    )
    return state, final


@partial(jax.jit, static_argnames=("nl_fn",))
def solve_sv_excitation(
    gamma2_mu,  # (n_modes,)
    omega_mu_squared,  # (n_modes,)
    modal_excitation: jnp.ndarray,  # (T, n_modes)
    dt: float,
    nl_fn: Callable,
):
    A_inv = A_inv_vector(dt, gamma2_mu)
    B = B_vector(dt, omega_mu_squared) * A_inv
    C = C_vector(dt, gamma2_mu) * A_inv

    n_modes = A_inv.shape[0]
    q = jnp.zeros((n_modes,))  # Modal displacement vector at n (n_modes, 1)
    q_prev = jnp.zeros((n_modes,))  # Modal displacement vector at n-1

    def advance_state(
        state: jnp.ndarray,  # inital state
        x: jnp.ndarray,  # input
    ) -> tuple[jnp.ndarray, jnp.ndarray]:  # carry, output
        q_prev, q = state
        nl = nl_fn(q)
        q_next = B * q + C * q_prev - A_inv * nl + A_inv * x
        return (q, q_next), q_next

    state, final = jax.lax.scan(
        advance_state,
        (q_prev, q),
        modal_excitation,  # (T, n_modes)
        unroll=8,
    )
    return state, final


@partial(
    jax.jit,
    static_argnames=(
        "n_steps",
        "nl_fn",
    ),
)
def solve_sv_initial_conditions(
    gamma2_mu,  # (n_modes,)
    omega_mu_squared,  # (n_modes,)
    u0: jnp.ndarray,  # initial conditions (n_modes,)
    v0: jnp.ndarray,  # initial conditions (n_modes,)
    dt: float,
    n_steps: int,
    nl_fn: Callable,
):
    A_inv = A_inv_vector(dt, gamma2_mu)
    B = B_vector(dt, omega_mu_squared) * A_inv
    C = C_vector(dt, gamma2_mu) * A_inv

    q0 = u0
    q1, _ = rk4_step(u0, v0, dt, gamma2_mu, omega_mu_squared)

    def advance_state(
        state: jnp.ndarray,  # inital state
        _: None,  # input
    ) -> tuple[jnp.ndarray, jnp.ndarray]:  # carry, output
        q_prev, q = state
        nl = nl_fn(q)
        q_next = B * q + C * q_prev - A_inv * nl
        return (q, q_next), q_next

    state, final = jax.lax.scan(
        advance_state,
        (q0, q1),
        length=n_steps - 2,
        unroll=8,
    )
    final = jnp.concatenate([q0[None], q1[None], final], axis=0)

    return state, final

## Discretised transfer function method

In [ ]:
# | export


@partial(jax.jit, static_argnames=("nl_fn",))
def solve_tf_excitation(
    gamma2_mu,
    omega_mu_squared,
    modal_excitation: jnp.ndarray,  # (T, n_modes)
    dt: float,
    nl_fn: Callable,
):
    """Solve using transfer-function (TF) based recurrence."""
    gamma_mu = gamma2_mu / 2.0
    omega_mu_damped = jnp.sqrt(omega_mu_squared - gamma_mu**2)
    radius = jnp.exp(-gamma_mu * dt)
    imag = radius * jnp.sin(omega_mu_damped * dt)
    real = radius * jnp.cos(omega_mu_damped * dt)

    b1_exc = dt * imag / omega_mu_damped

    a1 = 2.0 * real
    a2 = -(radius**2)

    n_modes = modal_excitation.shape[-1]
    q = jnp.zeros((n_modes,))  # Modal displacement vector at n (n_modes, 1)
    q_prev = jnp.zeros((n_modes,))  # Modal displacement vector at n-1

    def advance_state(
        state: jnp.ndarray,  # inital carry
        x: jnp.ndarray,  # input
    ) -> tuple[jnp.ndarray, jnp.ndarray]:  # carry, output
        q_prev, q_curr = state
        nl = nl_fn(q_curr)
        q_next = a1 * q_curr + a2 * q_prev - b1_exc * nl + b1_exc * x
        return (q_curr, q_next), q_next

    state, final = jax.lax.scan(
        advance_state,
        (q_prev, q),
        modal_excitation,
        unroll=8,
    )
    return state, final


@partial(
    jax.jit,
    static_argnames=(
        "n_steps",
        "nl_fn",
    ),
)
def solve_tf_initial_conditions(
    gamma2_mu,
    omega_mu_squared,
    u0: jnp.ndarray,  # initial conditions (n_modes,)
    v0: jnp.ndarray,  # initial conditions (n_modes,)
    dt: float,
    n_steps: int,
    nl_fn: Callable,
):
    """Solve using transfer-function (TF) based recurrence."""
    gamma_mu = gamma2_mu / 2.0
    omega_mu_damped = jnp.sqrt(omega_mu_squared - gamma_mu**2)
    radius = jnp.exp(-gamma_mu * dt)
    imag = radius * jnp.sin(omega_mu_damped * dt)
    real = radius * jnp.cos(omega_mu_damped * dt)

    b1_exc = dt * imag / omega_mu_damped

    a1 = 2.0 * real
    a2 = -(radius**2)

    q0 = u0
    q1, _ = rk4_step(u0, v0, dt, gamma2_mu, omega_mu_squared)

    def advance_state(
        state: jnp.ndarray,  # inital carry
        x: jnp.ndarray,  # input
    ) -> tuple[jnp.ndarray, jnp.ndarray]:  # carry, output
        q_prev, q_curr = state
        nl = nl_fn(q_curr)
        q_next = a1 * q_curr + a2 * q_prev - b1_exc * nl
        return (q_curr, q_next), q_next

    state, final = jax.lax.scan(
        advance_state,
        (q0, q1),
        length=n_steps - 2,
        unroll=8,
    )
    final = jnp.concatenate([q0[None], q1[None], final], axis=0)
    return state, final

## Sinusoidal solve

Solve the system of ODEs using complex exponentials

The system of ODEs is given by:

$$
\ddot{q} + 2 \gamma \dot{q} + \omega^2 q = 0
$$

where $\gamma$ is the damping coefficient and $\omega$ is the frequency. The damped frequencies are given by:

$$
\tilde{\omega} = \omega \sqrt{1 - \gamma^2} = \sqrt{\omega^2 - \gamma^2}
$$
The eigenvalues after the dispersion relation are then
$$
s_\pm = -\gamma \pm i \tilde{\omega}
$$

and discrete time eigenvalues (poles) are
$$
z_\pm = e^{s_\pm \Delta t} = e^{-\gamma \Delta t} e^{\pm i \tilde{\omega} \Delta t}
$$

In [ ]:
# | export


def solve_sinusoidal(
    gamma2_mu,
    omega_mu_squared,
    ic,
    n_steps,
    dt,
):
    """
    Solve the system of ODEs using complex exponentials
    NB: this assumes the ic is only for positions and that the initial velocities are 0

    Parameters
    ----------
    gamma2_mu : jnp.ndarray
        Damping coefficients
    omega_mu_squared : jnp.ndarray
        Squared frequencies
    ic : jnp.ndarray
        Initial conditions
    n_steps : int
        Number of steps
    dt : float
        Time step

    Returns
    -------
    jnp.ndarray
        Modal solution
    """
    gamma_mu = gamma2_mu / 2.0
    omega_mu = jnp.sqrt(omega_mu_squared - gamma_mu**2)
    s_mu = -gamma_mu + 1j * omega_mu
    z_mu = jnp.exp(s_mu * dt)

    z_mu_sequence = jnp.repeat(z_mu[None, :], n_steps - 1, axis=0)
    # include the initial condition
    z_mu_sequence = jnp.concatenate(
        [
            jnp.ones(
                shape=(1, gamma2_mu.shape[0]),
                dtype=gamma2_mu.dtype,
            ),
            z_mu_sequence,
        ],
        axis=0,
    )
    z_mu_sequence = jax.lax.associative_scan(
        jnp.multiply,
        z_mu_sequence,
        axis=0,
    )
    modal_sol = z_mu_sequence.real.T * ic[:, None]
    return modal_sol

In [ ]:
# | export


def solve_sinusoidal_excitation(
    gamma2_mu,
    omega_mu_squared,
    modal_excitation: jnp.ndarray,  # (T, n_modes)
    dt: float,
):
    """
    Solve the modal system with sinusoidal response for external excitation using parallel scan.

    Parameters
    ----------
    gamma2_mu : jnp.ndarray
        Damping coefficients (n_modes,)
    omega_mu_squared : jnp.ndarray
        Squared frequencies (n_modes,)
    modal_excitation : jnp.ndarray
        Modal excitation (T, n_modes)
    dt : float
        Time step

    Returns
    -------
    jnp.ndarray
        Modal solution (T, n_modes)
    """

    gamma_mu = gamma2_mu / 2.0
    omega_mu = jnp.sqrt(omega_mu_squared - gamma_mu**2)
    s_mu = -gamma_mu + 1j * omega_mu
    z_mu = jnp.exp(s_mu * dt)

    # Number of time steps and modes
    n_steps = modal_excitation.shape[0]
    n_modes = modal_excitation.shape[1]

    z_mu_sequence = jnp.repeat(z_mu[None, :], n_steps, axis=0)

    # Define binary operator for parallel scan
    def binary_op(elem_i, elem_j):
        a_i, b_i = elem_i
        a_j, b_j = elem_j
        return a_i * a_j, b_i * a_j + b_j

    _, output = jax.lax.associative_scan(
        binary_op,
        (z_mu_sequence, modal_excitation.astype(z_mu_sequence.dtype)),
        axis=0,
    )

    modal_solution = output.imag

    return modal_solution

In [ ]:
# | export


def solve_tf_ic(gamma2_mu, omega_mu_squared, ic, n_steps, dt, nl_fn):
    gamma_mu = gamma2_mu / 2.0
    omega_mu_damped = jnp.sqrt(omega_mu_squared - gamma_mu**2)

    radius = jnp.exp(-gamma_mu * dt)
    imag = radius * jnp.sin(omega_mu_damped * dt)
    real = radius * jnp.cos(omega_mu_damped * dt)

    b1_ic = imag / omega_mu_damped * gamma_mu - real

    b1_exc = dt * imag / omega_mu_damped

    b1_ic = -b1_ic * ic

    a1 = 2.0 * real
    a2 = -(radius**2)

    # initial condition: q0 = ic
    q0 = ic

    # compute q1 using Taylor expansion
    v0 = jnp.zeros_like(ic)
    ddq0 = -gamma2_mu * v0 - omega_mu_squared * ic
    q1 = ic + dt * v0 + 0.5 * dt**2 * ddq0

    # recurrence loop
    def step_fn(q_past, _):
        q_prev, q_curr = q_past

        nl = nl_fn(q_curr)

        q_next = a1 * q_curr + a2 * q_prev - b1_exc * nl
        return (q_curr, q_next), q_next

    (_, _), q_rest = jax.lax.scan(
        step_fn,
        (q0, q1),
        xs=None,
        length=n_steps - 2,
    )
    return jnp.concatenate([q0[:, None], q1[:, None], q_rest.T], axis=1)